## Task: Taking the Collision data and the Taxi data can we find correlation between accidents and the dropoff/pickup of taxis in the city?

### Which Borough has the most accidents? - Collision
### What are the victims break down (e.g. How many pedestrians, cyclist, ... were killed/injured in each borough? -  Collision

In [56]:
sc

In [32]:
!which python # Using personal envs since I'm on windows and trying to avoid conflicts with Python 2.7 and 3.5

/c/Anaconda3/envs/ipykernel_py2/python


In [3]:
import csv, pyproj, shapely.geometry as geom, numpy as np
proj = pyproj.Proj(init='epsg:2263', preserve_units=True)
COLLISIONS_FN = 'collisions.csv'
JANUARY_FN = '1january.csv'

In [4]:
collisons = sc.textFile(COLLISIONS_FN,use_unicode=False).cache()
january = sc.textFile(JANUARY_FN,use_unicode=False).cache()

In [47]:
print list(enumerate(collisons.first().split(',')))
# extract the number of collisions based on Boroughs with those that were killed and hurt.

[(0, 'DATE'), (1, 'TIME'), (2, 'BOROUGH'), (3, 'ZIP CODE'), (4, 'LATITUDE'), (5, 'LONGITUDE'), (6, 'LOCATION'), (7, 'ON STREET NAME'), (8, 'CROSS STREET NAME'), (9, 'OFF STREET NAME'), (10, 'NUMBER OF PERSONS INJURED'), (11, 'NUMBER OF PERSONS KILLED'), (12, 'NUMBER OF PEDESTRIANS INJURED'), (13, 'NUMBER OF PEDESTRIANS KILLED'), (14, 'NUMBER OF CYCLIST INJURED'), (15, 'NUMBER OF CYCLIST KILLED'), (16, 'NUMBER OF MOTORIST INJURED'), (17, 'NUMBER OF MOTORIST KILLED'), (18, 'CONTRIBUTING FACTOR VEHICLE 1'), (19, 'CONTRIBUTING FACTOR VEHICLE 2'), (20, 'CONTRIBUTING FACTOR VEHICLE 3'), (21, 'CONTRIBUTING FACTOR VEHICLE 4'), (22, 'CONTRIBUTING FACTOR VEHICLE 5'), (23, 'UNIQUE KEY'), (24, 'VEHICLE TYPE CODE 1'), (25, 'VEHICLE TYPE CODE 2'), (26, 'VEHICLE TYPE CODE 3'), (27, 'VEHICLE TYPE CODE 4'), (28, 'VEHICLE TYPE CODE 5')]


In [51]:
def extractConFacVeh(partID, records): # Location of Contributing Factors
    if partID==0:
        records.next()
    import csv
    reader = csv.reader(records)
    for row in reader:
        if len(row)==29:
            (boro,lat,lon,vehOne,vehTwo) = (row[2],row[4] ,row[5] ,row[19], row[20])
            yield (boro, lat, lon, vehOne,vehTwo)
            
pedResults = collisons.mapPartitionsWithIndex(extractConFacVeh)
pedResults.take(10)

[('BRONX', '40.866047', '-73.882744', 'Driver Inexperience', ''),
 ('BROOKLYN', '40.63347', '-74.02102', 'Unspecified', ''),
 ('BROOKLYN', '40.728416', '-73.952415', 'Unspecified', ''),
 ('QUEENS', '40.748928', '-73.91375', 'Unspecified', ''),
 ('QUEENS', '40.766445', '-73.83404', '', ''),
 ('', '40.843365', '-73.91545', 'Unspecified', ''),
 ('', '40.675785', '-73.90269', '', ''),
 ('BROOKLYN', '40.72516', '-73.9466', 'Unspecified', ''),
 ('BROOKLYN', '40.60771', '-73.95715', '', ''),
 ('BRONX', '40.8313', '-73.84458', 'Unspecified', '')]

In [66]:
def extractPedestrainsKilled(partID, records): # Pedestrains involved
    if partID==0:
        records.next()
    import csv
    reader = csv.reader(records)
    for row in reader:
        if row[12]!=0 and row[13]!=0:
            boro = row[2]
            #pedInj = int(row[12])
            pedKill = int(row[13])
            #(boro,lat,lon,pedInj,pedKill) = (row[2], row[4], row[5], row[12], row[13])
            yield (boro,pedKill)
            
def extractPedestrainsInjured(partID, records): # Pedestrains involved
    if partID==0:
        records.next()
    import csv
    reader = csv.reader(records)
    for row in reader:
        if row[12]!=0 and row[13]!=0:
            boro = row[2]
            pedInj = int(row[12])
            #pedKill = int(row[13])
            #(boro,lat,lon,pedInj,pedKill) = (row[2], row[4], row[5], row[12], row[13])
            yield (boro,pedInj)

In [67]:
pedResults0= collisons.mapPartitionsWithIndex(extractPedestrainsKilled).reduceByKey(lambda x,y: x+y)
pedResults0.take(10)

[('', 179),
 ('BRONX', 70),
 ('QUEENS', 137),
 ('STATEN ISLAND', 20),
 ('MANHATTAN', 119),
 ('BROOKLYN', 165)]

In [68]:
pedResults = collisons.mapPartitionsWithIndex(extractPedestrainsInjured).reduceByKey(lambda x, y: x + y)
pedResults.take(10)

[('', 7612),
 ('BRONX', 7238),
 ('QUEENS', 10710),
 ('STATEN ISLAND', 1321),
 ('MANHATTAN', 11822),
 ('BROOKLYN', 15897)]

In [72]:
def extractCyclistInjured(partID, records): # Cyclist
    if partID==0:
        records.next()
    import csv
    reader = csv.reader(records)
    for row in reader:
        if row[14]!=0:
            boro = row[2]
            ci = int(row[14])
            yield (boro, ci)
            
def extractCyclistKilled(partID, records): # Cyclist Injured by boro
    if partID==0:
        records.next()
    import csv
    reader = csv.reader(records)
    for row in reader:
        if row[15]!=0:
            boro = row[2]
            ck = int(row[15])
            yield (boro, ck)           

In [73]:
cycResults = collisons.mapPartitionsWithIndex(extractCyclistKilled).reduceByKey(lambda x, y: x + y)
cycResults.take(10)

[('', 16),
 ('BRONX', 7),
 ('QUEENS', 18),
 ('STATEN ISLAND', 2),
 ('MANHATTAN', 12),
 ('BROOKLYN', 21)]

In [74]:
cycResults = collisons.mapPartitionsWithIndex(extractCyclistInjured).reduceByKey(lambda x, y: x + y)
cycResults.take(10)

[('', 2885),
 ('BRONX', 1642),
 ('QUEENS', 3553),
 ('STATEN ISLAND', 186),
 ('MANHATTAN', 5423),
 ('BROOKLYN', 7190)]

In [82]:
def extractMotoristKilled(partID, records): # Motorist
    if partID==0:
        records.next()
    import csv
    reader = csv.reader(records)
    for row in reader:
        if row[17]!=0:
            boro = row[2]
            motKill = int(row[17])
            yield (boro,motKill)

def extractMotoristInjured(partID, records): # Motorist
    if partID==0:
        records.next()
    import csv
    reader = csv.reader(records)
    for row in reader:
        if row[16]!=0:
            boro = row[2]
            motKill = int(row[16])
            yield (boro,motKill)
            


In [80]:
pedResults = collisons.mapPartitionsWithIndex(extractMotoristKilled).reduceByKey(lambda x, y: x + y)
pedResults.take(10)

[('', 199),
 ('BRONX', 38),
 ('QUEENS', 85),
 ('STATEN ISLAND', 25),
 ('MANHATTAN', 20),
 ('BROOKLYN', 93)]

In [83]:
pedResults = collisons.mapPartitionsWithIndex(extractMotoristInjured).reduceByKey(lambda x, y: x + y)
pedResults.take(10)

[('', 66028),
 ('BRONX', 20231),
 ('QUEENS', 37643),
 ('STATEN ISLAND', 7058),
 ('MANHATTAN', 15954),
 ('BROOKLYN', 45979)]

In [8]:
print list(enumerate(january.first().split(',')))

[(0, 'pickup_datetime'), (1, 'pickup_latitude'), (2, 'pickup_longitude'), (3, 'trip_distance'), (4, 'passenger_count'), (5, 'dropoff_datetime'), (6, 'dropoff_latitude'), (7, 'dropoff_longitude'), (8, 'fare_amount'), (9, 'tolls_amount'), (10, 'taxes_amount'), (11, 'tip_amount'), (12, 'payment_amount'), (13, 'payment_type')]


In [42]:
# We can take the long/lat of how many passengers were picked up
def extractPickUp(partID, records):
    if partID==0:
        records.next()
    import csv
    reader = csv.reader(records)
    for row in reader:
        if len(row)==14:
            (pLat,pLong,pass_count) = (row[1],row[2],int(row[4]))
            yield (pLat,pLong,pass_count)
            
passResults = january.mapPartitionsWithIndex(extractPickUp)
passResults.take(5)

[('40.7347', '-73.99037', 2),
 ('40.72991', '-73.98078', 5),
 ('40.67957', '-73.98455', 1),
 ('40.71899', '-73.99347', 1),
 ('40.78133', '-73.96062', 3)]

In [45]:
def extractDropOff(partID, records):
    if partID==0:
        records.next()
    import csv
    reader = csv.reader(records)
    for row in reader:
        if len(row)==14:
            (dLat,dLong,pass_count) = (row[6],row[7],int(row[4]))
            yield (dLat,dLong,pass_count)
            
passResults = january.mapPartitionsWithIndex(extractDropOff)
passResults.take(5)

[('40.73241', '-73.98184', 2),
 ('40.71668', '-73.94447', 5),
 ('40.78893', '-73.95027', 1),
 ('40.65733', '-73.96224', 1),
 ('40.75851', '-73.97726', 3)]